### Model Test
Before we move onto tweaking and tuning models, we should ensure that our hardware is working as anticipated.

One of the major turnoffs to machine learning on vast data is the amount of time it can take to train. Using your device's GPU can significantly reduce image processing time and greatly reduce headaches (and the temperature of your CPU).

For this exercise, I encourage using a smaller portion of the project data. However, if you're confident using [TensorFlow](https://www.tensorflow.org/), you might as well go for all of it. As you might've guessed, TensorFlow isn't the easiest Python library to manage. You'll need more than a simple `pip install tensorflow` to get it running.

Here is an exceptionally helpful [tutorial](https://towardsdatascience.com/installing-tensorflow-with-cuda-cudnn-and-gpu-support-on-windows-10-60693e46e781) to get you started. It's tailored to Windows 10 and NVIDIA, but it should give you a good starting point even if you don't use that setup.

The setup might be painful, but I promise it's worth it. The cell below contains code that, when run, list out all of the devices that TensorFlow is using. On most setups, a CPU and GPU should appear (depending on your hardware)


#### Notebook Goals
- Enable TensorFlow to work on your machine as optimally as possible.
- Test the pathing schema of our training data.
- Run this notebook from top to bottom, ensuring no errors exist.

In [ ]:
# The output shows the list of devices you're using. For me, that's my CPU and two default GPUs, as shown.
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

``` 
name: "/cpu:0" device_type: "CPU" memory_limit: 268435456 locality { } incarnation: 4402277519343584096,

name: "/gpu:0" device_type: "GPU" memory_limit: 6772842168 locality { bus_id: 1 } incarnation: 7471795903849088328 physical_device_desc: "device: 0, name: GeForce RTX 2080ti, pci bus id: 0000:05:00.0"

name: "/gpu:1" device_type: "GPU" memory_limit: 6772842168 locality { bus_id: 2 } incarnation: 74717959038490889912 physical_device_desc: "device: 0, name: GeForce RTX 2070, pci bus id: 0000:05:00.0"

```

Your output will be different from mine. Ensure you have a GPU available (if you have one) before moving onto the next cells.


#### Staging your data
The code block below deals with staging our data for processing and class definitions. If you compare this to many "tutorial" versions of tensorflow modelling, you'll find that this method is much easier on your computer's hardware. Why?

*Don't worry about the processing procedures too much; this notebook is meant to tell us if TensorFlow is working properly for you. We'll tune this later.*

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

# You can point this to one folder above your class-defined folders
train_folder = 'path\\to\\train\\'
val_folder = 'path\\to\\valid\\'

batch_size = 32


train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

# This method depends on our folder schema that we established in earlier phases of the project
train_generator = train_datagen.flow_from_directory(train_folder,
                                                    color_mode='grayscale',
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(val_folder,
                                                        color_mode='grayscale',
                                                        batch_size=batch_size,
                                                        class_mode='categorical')

#### Preparing your model
The model below is far from optimized. Again, this notebook is only supposed to tell us if your setup is working.

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=3, activation='relu', padding='same', input_shape=(256, 256, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

#Flatten the model and prepare for output
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax')) #softmax works for categorical

# Compile model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

#### Training your model
The following might take a whole to run.

In [ ]:
history = model.fit_generator(train_generator,
                              steps_per_epoch=20548 // batch_size,
                              epochs=2,
                              verbose=1,
                              validation_data=validation_generator,
                              validation_steps=5136 // batch_size)

#### Congratulations! You should be ready to make this model more complex now.